In [1]:
#API_KEY ALPHA_VANTAGE
api_key = '343SXIODK7XQVG9D'

In [2]:
#CODE TO DOWNLOAD CSV FILE FROM YAHOO FINANCE
import requests
import yfinance as yf
import csv
import os
from datetime import datetime
import sys  # Import the sys module
import pandas as pd

# Set the maximum number of allowed incorrect inputs
max_incorrect_inputs = 5

# Initialize the incorrect input counter
incorrect_input_count = 0

# Function to validate the date format
def get_valid_date_input(prompt):
    while True:
        date_str = input(prompt)
        try:
            date_obj = datetime.strptime(date_str, "%Y-%m-%d")
            return date_obj
        except ValueError:
            global incorrect_input_count
            incorrect_input_count += 1
            if incorrect_input_count >= max_incorrect_inputs:
                print("Too many incorrect inputs. Exiting program.")
                sys.exit()  # Terminate the program
            print("Error: Invalid date format. Please use the format YYYY-MM-DD.")
            print(f"Remaining tries: {max_incorrect_inputs - incorrect_input_count}")

# Function to validate the timeframe
def get_valid_timeframe():
    valid_timeframes = ["1d", "1wk", "1mo"]
    while True:
        timeframe = input("Enter the timeframe (1d, 1wk, or 1mo): ").strip()
        if timeframe in valid_timeframes:
            return timeframe
        else:
            global incorrect_input_count
            incorrect_input_count += 1
            if incorrect_input_count >= max_incorrect_inputs:
                print("Too many incorrect inputs. Exiting program.")
                sys.exit()  # Terminate the program
            print("Error: Invalid timeframe. Please enter 1d, 1wk, or 1mo.")
            print(f"Remaining tries: {max_incorrect_inputs - incorrect_input_count}")

# Prompt the user for a stock symbol
user_input_stock_symbol = input("Enter a stock symbol: ").strip().upper()

# Prompt the user for the start and end dates
start_date = get_valid_date_input("Enter the start date (YYYY-MM-DD): ")
end_date = get_valid_date_input("Enter the end date (YYYY-MM-DD): ")

# Check if the start date is prior to the end date
if start_date > end_date:
    print("Error: Start date must be prior to the end date. Exiting program.")
    sys.exit()  # Terminate the program

# Prompt the user for the timeframe
timeframe = get_valid_timeframe()

# Use an f-string to interpolate the user input and API key into the URL
url = f'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords={user_input_stock_symbol}&apikey={api_key}'

# Send an HTTP GET request to the Alpha Vantage symbol search API
response = requests.get(url)

# Parse the JSON response
data = response.json()

# Check if data contains results
if "bestMatches" in data:
    results = data["bestMatches"]

    if results:
        # Extract the symbol of the first best match
        symbol = results[0]["1. symbol"]

        # Get the current working directory
        current_directory = os.getcwd()

        # Delete any CSV files in the current directory
        for filename in os.listdir(current_directory):
            if filename.endswith(".csv"):
                file_path = os.path.join(current_directory, filename)
                os.remove(file_path)
                print(f'Removed old file: {filename}')

        # Construct the CSV filename based on the user's input stock symbol
        csv_filename = f'{symbol}_historical_data.csv'
        csv_full_path = os.path.join(current_directory, csv_filename)

        # Download data using yfinance with user-defined dates and timeframe
        stock_data = yf.download(symbol, start=start_date, end=end_date, period=timeframe)

        # Save the data to the dynamically named CSV file
        stock_data.to_csv(csv_full_path)
        stock_pd = pd.read_csv('SPY_historical_data.csv')
        stock_pd.pop('Adj Close')
        stock_pd.pop('Volume')        

        print(f'Data for {symbol} from {start_date.strftime("%Y-%m-%d")} to {end_date.strftime("%Y-%m-%d")} with a {timeframe} timeframe saved to {csv_full_path}')
    else:
        print("No results found for the given stock symbol.")
else:
    print("Invalid response from Alpha Vantage.")


Removed old file: SPY_historical_data.csv
[*********************100%%**********************]  1 of 1 completed
Data for SPY from 2022-10-08 to 2023-10-08 with a 1d timeframe saved to c:\Projects\Project Tradezodia\SPY_historical_data.csv


In [8]:
# Add indicators - MACD
def calculate_macd(stock_data):
    stock_data['EMA_12'] = stock_data['Close'].ewm(span=12, adjust=False).mean()
    stock_data['EMA_26'] = stock_data['Close'].ewm(span=26, adjust=False).mean()
    stock_data['MACD'] = stock_data['EMA_12'] - stock_data['EMA_26']
    stock_data['Signal_Line'] = stock_data['MACD'].ewm(span=9, adjust=False).mean()
    stock_data['MACD_Histogram'] = stock_data['MACD'] - stock_data['Signal_Line']
calculate_macd(stock_data)



In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

# Assume you have already calculated the MACD values in your stock_data DataFrame

# Create subplots with 2 rows and 1 column
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.2, row_heights=[0.7, 0.3])

# Add Candlestick Chart to the first subplot (row 1)
fig.add_trace(go.Candlestick(x=stock_data.index,
                             open=stock_data['Open'],
                             high=stock_data['High'],
                             low=stock_data['Low'],
                             close=stock_data['Close']), row=1, col=1)

fig.update_layout(
    title=f'{timeframe} Candlestick Chart for {symbol} from {start_date} to {end_date}',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=True,
    yaxis=dict(autorange=True, fixedrange=False)
)

# Add MACD indicators to the second subplot (row 2)
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['MACD'], mode='lines', name='MACD'), row=2, col=1)
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Signal_Line'], mode='lines', name='Signal Line'), row=2, col=1)
fig.add_trace(go.Bar(x=stock_data.index, y=stock_data['MACD_Histogram'], name='MACD Histogram'), row=2, col=1)

fig.update_layout(
    yaxis2=dict(title='MACD'),
)

fig.write_html("candlestick_chart_with_macd.html")
